In [1]:
import pandas as pd
import numpy as np
from pvlib import location, irradiance, temperature, pvsystem
import matplotlib.pyplot as plt
from random import seed
from itertools import product
from sandiapv_energy_prediction import *


C:\Users\denni\AppData\Local\Temp\ipykernel_17116\4101271371.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
def sensitivity_analysis(weather_data, site_location, tilt_range, azimuth_range, module_types):
    results = []
    for tilt, azimuth, module_type in product(tilt_range, azimuth_range, module_types):
        panel = {'type': 'Canadian_Solar_CS5P_220M___2009_', 'tilt': tilt, 'azimuth': azimuth, 'module_type': module_type}
        system = get_pv_system(panel)
        output = simulate_pv_output(system, weather_data, site_location)
        results.append({'tilt': tilt, 'azimuth': azimuth, 'module_type': module_type, 'output': output.sum().sum()})
    return pd.DataFrame(results)

In [8]:
weather_data = load_weather_data('energy_data/historical_weather.csv')
weather_data.index = pd.to_datetime(weather_data['timestamp'], utc=True)
site_location = location.Location(latitude=52.52, longitude=13.4050, altitude=34, tz='Europe/Amsterdam')
    
# Define ranges for parameters
tilt_range = np.linspace(5, 40, 10)  
azimuth_range = np.linspace(0, 360, 36)  
module_types = ['monocrystalline', 'polycrystalline', 'thin-film', 'bifacial']

# Perform sensitivity analysis
sa_results = sensitivity_analysis(weather_data, site_location, tilt_range, azimuth_range, module_types)
    
# Save results to CSV
sa_results.to_csv('energy_data/sensitivity_analysis_results.csv', index=False)

KeyError: 'T'

In [ ]:
# Plotting results
for module_type in module_types:
    plt.figure()
    subset = sa_results[sa_results['module_type'] == module_type]
    for tilt in tilt_range:
        data = subset[subset['tilt'] == tilt]
    plt.plot(data['azimuth'], data['output'], label=f'Tilt {tilt}°')
    plt.title(f'PV Output vs. Azimuth for {module_type} Modules')
    plt.xlabel('Azimuth (degrees)')
    plt.ylabel('Total Output')
    plt.legend()
    plt.grid(True)
    plt.show()